# Исследование надежности заемщиков.

**Цель исследования**

Необходимо разобраться, влияет ли семейное положение, количество детей, доход и цели займа на факт погашения кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга. 

Ответить на вопросы/проверить гипотезы:
1. Есть ли зависимость между **количеством детей** и возвратом кредита в срок?
2. Есть ли зависимость между **семейным положением** и возвратом кредита в срок?
3. Есть ли зависимость между **уровнем дохода** и возвратом кредита в срок?
4. Как разные **цели** кредита влияют на его возврат в срок?

**Ход исследования**

Входные данные от банка — статистика о платёжеспособности клиентов из файла `data.csv`.
 
Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Обработка данных.
 3. Проверка гипотез.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
import os

In [2]:
pth1 = '/Users/karter/Projects/1_borrower_reliability_studies.ipynb/data.csv'
pth2 = 'data.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth2):
    data = pd.read_csv(pth2)
else:
    print('Something is wrong')

data.info()
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
display(data['education'].unique()) # Все уюникальные значения столбца образования

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [4]:
display(data['family_status_id'].unique())# Все уюникальные значения категорий семейного положения

array([0, 1, 2, 3, 4])

In [5]:
display(data['family_status'].unique())# Все уюникальные значения столбца семейное положение

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [6]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [7]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [5]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [6]:
data.sort_values(by='days_employed', ascending = False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба


In [7]:
data.groupby('income_type')['days_employed'].sum()

income_type
безработный        7.328273e+05
в декрете         -3.296760e+03
госслужащий       -4.460665e+06
компаньон         -9.664447e+06
пенсионер          1.256707e+09
предприниматель   -5.208481e+02
сотрудник         -2.329756e+07
студент           -5.787516e+02
Name: days_employed, dtype: float64

В таблице 12 столбцов
Согласно документации к данным:
    
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income`— ежемесячный доход
* `purpose` — цель получения кредита

В названиях колонок нарушений стиля не выявлено.

Количество значений в столбце days_employed и total_income отличается от остольных и имеют 19351, 
в остальных по 21525.

В столбце education встречаются значения с верхним регистром.

В столбец days_employed есть положительные значения у пенсионеров и безработных, отрицательные значения скорее всего указаны по ошибке. Если перевести все значения к знаку +, то можем увидеть что у пенсионеров слишком много дней, могу предположить что у пенсионеров указаны данные не в днях, а в часах.

### Шаг 2.1 Заполнение пропусков

In [8]:
employed = data['days_employed'].isna().sum()
education = data['education'].count()
display(f'Доля пропусков: {int(employed / education * 100)}%')

'Доля пропусков: 10%'

In [9]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропущены были значения в столбцах days_employed и total_income, пропусков было равное колличество, во все остальных столбцах пропущенных значений не было;

Пропущеных значений было 10% от общего числа;

Возможные причины пропущенных значений это ошибки в заполнении данных или отсутвии стажа работы и соответственно зароботка;

Заполнил пропуски медианным значеним так как в количественных переменных среднее значение может не меняться при большом разбросе значений.

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [10]:
data['days_employed'] = data['days_employed'].abs()
data.loc[data['days_employed'] > 0, 'days_employed']=(data['days_employed']/24)
data['children'] = data['children'].abs()
display(data.head(50))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,167.700156,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234.309275,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,171.864467,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,38.591076,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,119.966752,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,6.365815,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,288.744387,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,91.198185,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [11]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351.569709,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,167.700156,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234.309275,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,171.864467,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,188.721528,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,14330.725172,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,88.056120,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,129.686738,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


При обзоре данных было замечено что в столбце days_employed есть отрицательные и завышенные значения. Исправил значения, если значение больше 0, такие как у пенсионер(завышены) переводим из часов в дни, так же исправил отрицательные значение на положительные, теперь данные выглядят более правдоподобно.

Есть аномальные значения в столбце cildren, отрицательных значений не много, и только -1, их перевел в положитльные. У некотрых подозрительно много детей, при категоризации отнесем их к многодетным.

### Шаг 2.3. Изменение типов данных.

In [12]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


В обзоре данных было видно что в стобце total_income вещественный тип данных, сменил на целочисленный для дальнейшей обработке данных.

### Шаг 2.4. Удаление дубликатов.

In [13]:
data['education'] = data['education'].str.lower()

In [14]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [15]:
display(data[data.duplicated()].head())
display(data[data.duplicated()].tail())
display(f' Дубликатов: {data.duplicated().sum()}')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,50.140397,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,50.140397,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,50.140397,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,50.140397,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,50.140397,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20702,0,50.140397,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
21032,0,50.140397,60,среднее,1,женат / замужем,0,F,пенсионер,0,118514,заняться образованием
21132,0,50.140397,47,среднее,1,женат / замужем,0,F,сотрудник,0,142594,ремонт жилью
21281,1,50.140397,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости
21415,0,50.140397,54,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции с жильем


' Дубликатов: 71'

In [16]:
data = data.drop_duplicates().reset_index(drop=True) 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


В столбце education были значения с верхним и нижним регистром, перед удаление дубликатов привел все значения к нижнему регистру. 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [17]:
education = data[['education','education_id']]
family_status = data[['family_status','family_status_id']]
data = data.drop(columns = ['education', 'family_status'])

Вынесли в отдельные таблицы те столбцы у которых есть категоризация, для упрощения таблицы

### Шаг 2.6. Категоризация дохода.

In [18]:
def income_group(total):
    if total >= 0 and total <= 30000:
        return 'E'
    if total >= 30001 and total <= 50000:
        return 'D'
    if total >= 50001 and total <= 200000:
        return 'C'
    if total >= 200001 and total <= 1000000:
        return 'B'
    if total >= 1000001:
        return 'A'
data['total_income_category'] = data['total_income'].apply(income_group)
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,351.569709,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,167.700156,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,234.309275,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,171.864467,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,38.591076,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,119.966752,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,6.365815,50,1,0,M,сотрудник,0,135823,образование,C
8,2,288.744387,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,91.198185,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


In [19]:
data['total_income_category'].isna().sum()

0

Категории:

A - доход более 1000001

B - доход от 200001 до 1000000

C - доход от 50001 до 200000

D - доход от 30001 до 50000

E - доход менее 30000

### Шаг 2.7. Категоризация целей кредита.

In [20]:
def purpose_group(purpose):
    try:
        if 'автомоб' in purpose:
            return 'операции с автомобилем'       
        if 'недвижим' in purpose or 'жиль'in purpose:
            return 'операции с недвижимостью'
        if 'образован' in purpose:
            return 'получение образования'       
        if 'свадьб' in purpose:
            return 'проведение свадьбы'
    except:
        print('нет совпадений')  
data['purpose_category'] = data['purpose'].apply(purpose_group)
display(data.head(20))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,351.569709,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,167.700156,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,234.309275,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,171.864467,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,38.591076,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,119.966752,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,6.365815,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,288.744387,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,91.198185,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21454 non-null  int64  
 1   days_employed          21454 non-null  float64
 2   dob_years              21454 non-null  int64  
 3   education_id           21454 non-null  int64  
 4   family_status_id       21454 non-null  int64  
 5   gender                 21454 non-null  object 
 6   income_type            21454 non-null  object 
 7   debt                   21454 non-null  int64  
 8   total_income           21454 non-null  int64  
 9   purpose                21454 non-null  object 
 10  total_income_category  21454 non-null  object 
 11  purpose_category       21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Категоризировал данные по доходу, назначению кредита.

### Шаг 3. Проверка гипотез.

#### Влияние наличие детей на факт погашения кредита в срок

In [22]:
debt_children = pd.DataFrame()
debt_children['count_children'] = data.groupby('children')['debt'].count()
debt_children['sum_children'] = data.groupby('children')['debt'].sum()
debt_children['result_children'] = debt_children['sum_children'] / debt_children['count_children'] 
debt_children.sort_values('result_children', ascending = False)

,count_children,sum_children,result_children
children,,,
20,76,8,0.105263
4,41,4,0.097561
2,2052,194,0.094542
1,4855,445,0.091658
3,330,27,0.081818
0,14091,1063,0.075438
5,9,0,0.000000


Тут мы можем видеть что максимальная доля просрочек у сеймей с 20 детьми.

#### Влияние семейного положение клиента на факт погашения кредита в срок

In [23]:
debt_family = pd.DataFrame()
debt_family['count_family'] = data.groupby('family_status_id')['debt'].count()
debt_family['sum_family'] = data.groupby('family_status_id')['debt'].sum()
debt_family['result_family'] = debt_family['sum_family'] / debt_family['count_family'] 
debt_family.sort_values('result_family', ascending = False)

,count_family,sum_family,result_family
family_status_id,,,
4,2810,274,0.097509
1,4151,388,0.093471
0,12339,931,0.075452
3,1195,85,0.071130
2,959,63,0.065693


0 - женат / замужем

1 - гражданский брак

2 - вдовец / вдова

3 - в разводе

4 - Не женат / не замужем


Тут мы можем видеть что максимальную долю просрочек думаскают клиенты находящиеся в браке 

#### Влияние дохода клиента на факт погашения кредита в срок

In [24]:
debt_income = pd.DataFrame()
debt_income['count_income'] = data.groupby('total_income_category')['debt'].count()
debt_income['sum_income'] = data.groupby('total_income_category')['debt'].sum()
debt_income['result_income'] = debt_income['sum_income'] / debt_income['count_income'] 
debt_income.sort_values('result_income', ascending = False)

,count_income,sum_income,result_income
total_income_category,,,
E,22,2,0.090909
C,16015,1360,0.084920
A,25,2,0.080000
B,5042,356,0.070607
D,350,21,0.060000


A - доход более 1000001

B - доход от 200001 до 1000000

C - доход от 50001 до 200000

D - доход от 30001 до 50000

E - доход менее 30000

Самая высокая доля просрочек у клиентов с минимальным доходом, менее 30000, но есть интересное наблюдение, минимальная доля просрочек у клиентов с доходом на одну каотегорию выше, тоесть от 30000 до 50000. Так же клиенты с достаточно высоким доходом допускают большое количество просрочек.

#### Влияние цели кредитования клиента на факт погашения кредита в срок

In [25]:
debt_purpose = pd.DataFrame()
debt_purpose['count_purpose'] = data.groupby('purpose_category')['debt'].count()
debt_purpose['sum_purpose'] = data.groupby('purpose_category')['debt'].sum()
debt_purpose['result_purpose'] = debt_purpose['sum_purpose'] / debt_purpose['count_purpose'] 
debt_purpose.sort_values('result_purpose', ascending = False)

,count_purpose,sum_purpose,result_purpose
purpose_category,,,
операции с автомобилем,4306,403,0.093590
получение образования,4013,370,0.092200
проведение свадьбы,2324,186,0.080034
операции с недвижимостью,10811,782,0.072334


Клиенты взявшие кредит на покупку автомобиля имеют самую высокоую долю просрочек.

### Ответы на вопросы.

##### Вопрос 1:

Влияет ли наличии детей на факт погашения кредита в срок?

##### Вывод 1:

Клиенты с 20 детьми допускают больше всего просрочек.

##### Вопрос 2:

Влияет ли семейное положение клиента на факт погашения кредита в срок

##### Вывод 2:

Клиенты браке допускают больше всего просрочек

##### Вопрос 3:

Есть ли зависимость от дохода на факт погашения кредита в срок?

##### Вывод 3:

Клиенты с доходом менее 30000 допускают больше всего просрочек

##### Вопрос 4:

Влияет ли цель кредита на факт погашения кредита в срок?

##### Вывод 4:

Клиенты взявшие кредит на покупку автомобиля допускают больше всего просрочек

## Общий вывод:

Семья (количество детей и семейное положение), уровень дохода и цель кредита безусловно влияют на возврат кредита в срок. Некоторые результаты оказались особенно интересными, в частности высокий уровень дохода вовсе не гарантирует возврат кредита в срок. Также показатели возврата кредита в срок значительно выше у вдовцов и вдов, нежели у состоящих или не состоящих в браке клиентов.
Основываясь на полученных данных мы приходим к следующему выводу:
Лучшие клиенты (относительно вероятности выплат кредита в срок): бездетные вдовцы/вдовы с доходом от 30001 до 50000 с целью кредита - операция с недвижимостью.
Худшие клиенты (относительно вероятности выплат кредита в срок): 4 (либо 2) ребенка, не женат / не замужем (либо гражданский брак), доход до 30000 (либо от 50001 до 200000), операции с автомобилем (либо получение образования).